In [1]:
! pip install pydrive

# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
from google.colab import files

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from keras.models import load_model, Model
from keras.models import model_from_json
from  keras import backend as K
from sklearn.model_selection import train_test_split

def prepare_dataset(dataframe, image_size={'width':48, 'height':48}):
    faces_images = []
    pixels = dataframe['pixels'].tolist()
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(image_size['width'], image_size['height'])
        face = cv2.resize(face.astype('uint8'), (image_size['width'], image_size['height']))
        
        faces_images.append(face.astype('float32') / 255.0)
    faces_images = np.asarray(faces_images)
    faces_images = np.expand_dims(faces_images, -1) # (1, 48, 48)
    emotions_labels = pd.get_dummies(dataframe['emotion']).as_matrix()
    return faces_images, emotions_labels

def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 0.5

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


drive.CreateFile({'id': "1ag2GpLbUY1YodSzsO-I-rZpp--D3VabW"}).GetContentFile('model.json')
drive.CreateFile({'id': "13xc-Y2Md3phQcdVhrn9-d9M7iUdIMnbS"}).GetContentFile('model.h5')
# drive.CreateFile({'id': "1zVWFGUKLBblOT1m3zxafcz-Yc3clEgUp"}).GetContentFile('model.hdf5')
drive.CreateFile({'id': '1Yln-m9Fuo4fIJ0enTHUSw-Nf7Fq7gA1_'}).GetContentFile('dataset.csv')

In [0]:
# load dataset from csv file
fer2013 = pd.read_csv("dataset.csv")
emotions_labels_map = {'angry':0,'disgust':1,'fear':2,'happy':3,'sad':4,'surprise':5,'neutral':6}
groups = fer2013.groupby('Usage')

training_data = groups.get_group('Training')
training_emotions = training_data.groupby('emotion')
test_data = groups.get_group('PublicTest')
test_emotions = test_data.groupby('emotion')
validation_data = groups.get_group('PrivateTest')
validation_emotions = validation_data.groupby('emotion')

# prepare dataset to training, test and validate model
training_faces, training_emotions = prepare_dataset(training_data)
test_faces, test_emotions = prepare_dataset(test_data)
validation_faces, validation_emotions = prepare_dataset(validation_data)

# get size dataset elements
num_samples, num_classes = training_emotions.shape

# split and randomize data
x_train_part1, x_train_part2, y_train_part1, y_train_part2 = train_test_split(training_faces, training_emotions, test_size=0.3, random_state=42)
x_train, y_train = np.vstack((x_train_part1, x_train_part2)), np.vstack((y_train_part1, y_train_part2))

x_test_part1, x_test_part2, y_test_part1, y_test_part2 = train_test_split(test_faces, test_emotions, test_size=0.3, random_state=1)
x_test, y_test = np.vstack((x_test_part1, x_test_part2)), np.vstack((y_test_part1, y_test_part2))

x_validation_part1, x_validation_part2, y_validation_part1, y_validation_part2 = train_test_split(validation_faces, validation_emotions, test_size=0.3, random_state=42)
x_validation, y_validation = np.vstack((x_validation_part1, x_validation_part2)), np.vstack((y_validation_part1, y_validation_part2))

# all data randomized
array_x = np.vstack((x_train, x_test))
array_y = np.vstack((y_train, y_test))

In [7]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("model.h5")

# model = load_model('model.hdf5')#, custom_objects={'fbeta': fbeta})

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 48, 48, 128)       6400      
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 128)       802944    
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
average_pooling2d_1 (Average (None, 24, 24, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       409728    
__________

In [0]:
# y_pred = model.predict_classes(array_x)

In [0]:
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('flatten_1').output)
intermediate_layer_model2 = Model(inputs=model.input, outputs=model.get_layer('dropout_6').output)

In [0]:
y_intermediate = intermediate_layer_model.predict(array_x)
y_intermediate2 = intermediate_layer_model2.predict(array_x)
y_true = np.asarray([np.argmax(i) for i in array_y])
y_intermediate = np.hstack((y_intermediate, y_intermediate2))

In [0]:
cols = ['a' + str(i) for i in range(len(y_intermediate[0]))]
df = pd.DataFrame(y_intermediate, columns=cols)
df['output'] = y_true

In [0]:
df.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a62,a63,a64,a65,a66,a67,a68,a69,a70,output
0,0.836700,0.000000,0.000000,0.000000,0.215083,0.000000,0.171599,0.000000,0.000000,0.328592,...,0.180276,0.000000,0.707058,0.001684,0.103417,0.167306,0.011203,0.008367,0.000964,0
1,0.000000,0.000000,0.000000,1.766487,0.220245,1.570082,0.000000,1.577820,1.692561,0.000000,...,0.000000,1.787054,0.041479,0.000240,0.072205,0.006838,0.456493,0.001418,0.421327,4
2,0.000000,0.000000,0.119241,0.000000,0.000000,0.000000,0.460938,0.000000,0.000000,0.000000,...,0.607736,0.000000,0.006486,0.000171,0.866016,0.015266,0.023661,0.085363,0.003036,2
3,0.000000,1.593817,0.739128,0.072315,0.943397,0.645937,0.000000,1.125072,0.000000,0.000000,...,0.371986,0.533491,0.228890,0.017975,0.160411,0.017499,0.192107,0.045088,0.338029,0
4,1.768697,0.251780,0.000000,0.000000,0.595878,0.000000,0.000000,0.000000,0.000000,0.211912,...,0.655107,0.000000,0.894587,0.003621,0.050616,0.038457,0.009549,0.002106,0.001064,3


In [0]:
df.to_csv('../datasets/train.csv')

In [0]:
y_pred = model.predict_classes(x_validation)

In [0]:
y_intermediate = intermediate_layer_model.predict(x_validation)
y_true = np.asarray([np.argmax(i) for i in y_validation])

In [0]:
cols = ['a' + str(i) for i in range(len(y_intermediate[0]))]
df = pd.DataFrame(y_intermediate, columns=cols)
df['output'] = y_true

In [0]:
df.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a55,a56,a57,a58,a59,a60,a61,a62,a63,output
0,0.000000,0.092281,0.000000,0.456152,0.191803,0.962075,0.000000,1.977278,0.600408,0.000000,...,0.855805,0.000000,0.786181,0.000000,0.000000,0.000000,0.000000,0.088484,0.921997,4
1,0.000000,1.503560,0.620297,0.000000,0.933554,0.000000,0.784707,0.000000,0.000000,0.000000,...,2.896611,0.000000,1.491447,0.398511,0.000000,0.000000,2.207073,2.876589,0.000000,5
2,1.351697,0.000000,0.498356,1.474676,0.000000,0.597703,1.153067,0.234085,1.662123,1.612338,...,0.000000,0.032306,0.000000,1.084711,0.000000,2.028031,0.487058,0.000000,1.438949,3
3,0.260849,0.000000,0.058829,3.512513,0.084941,1.901543,0.000000,1.214096,2.883028,0.000000,...,0.000000,0.346011,1.289381,0.000000,0.617684,1.747490,0.000000,0.000000,2.414159,6
4,0.000000,1.420389,0.591298,0.000000,1.102591,0.000000,0.679130,0.000000,0.000000,0.000000,...,0.953143,0.170608,0.251804,0.702773,0.369346,0.000000,0.549121,1.277051,0.000000,2


In [0]:
df.to_csv('../datasets/test.csv')